<a href="https://colab.research.google.com/github/ad17171717/YouTube-Tutorials/blob/main/Google%20Colab%20Tutorials/Can't_Run_Docker_in_Google_Colab%3F!_Try_udocker!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Why Docker Engine Does Not Work in Google Colab (for now)**

**Docker Engine is an open-source containerization technology for building and containerizing applications. To build and manage Docker containers, the user needs kernel-level access and full control of system resources. Full control is necessary because Docker relies on Linux kernel features such as Control Groups (cgroups) and Namespaces to create and manage containers.**

- **cgroups provide a mechanism in Linux for managing system resources. cgroups enable fine-grained control of resources such as CPU, memory, block I/O, and network bandwidth for Linux containers. Docker uses cgroups to ensure that each container has the resources it needs without impacting other containers or processes on the host system.**

- **Namespaces are used to isolate parts of the operating system. Docker creates a set of namespaces for each container to provide process isolation. This means that different aspects of the container, such as networking, file systems, and process IDs, are restricted to that container's namespaces and cannot access resources outside of them.**

**In the current Google Colab environment, users cannot configure or edit cgroups or Namespaces. This restriction exists because Google Colab runs in a shared environment where users do not have kernel-level privileges. Without control over these features, it is not possible to create or run Docker containers. As of October 2024, Google Colab does not support running Docker Engine. In the future, changes to Colab’s environment might allow users to run containers, but for now, these features remain unavailable.**

<sup>Source: [What is Docker?](https://docs.docker.com/get-started/docker-overview/) from docs.docker.com</sup>

<sup>Source: [Runtime metrics](https://docs.docker.com/engine/containers/runmetrics/) from docs.docker.com</sup>

<sup>Source: [CGROUPS](https://www.kernel.org/doc/Documentation/cgroup-v1/cgroups.txt) from kernel.org</sup>

### **Control Group (cgroup) limitations**

**In a Google Colab session, it is possible to create a cgroup; however, there are limited privileges to fully manage or modify cgroups.**

**Even though the default user in a Google Colab session is the `root` user, the `root` user’s permissions are restricted. Specifically, `root` does not have full read-write permissions to the `cgroup` directory. Attempts to modify the cgroup, such as writing to resource management files are denied due to insufficient privileges.**

**As a result, Docker, which relies on cgroups to manage resources for containers, cannot function properly in this environment if a cgroup cannot be modified.**

In [1]:
#check the current user
!whoami

root


In [2]:
#check that cgroups are limited to read only "ro"
!mount | grep /sys

sysfs on /sys type sysfs (ro,nosuid,nodev,noexec,relatime)
cgroup on /sys/fs/cgroup type cgroup2 (ro,nosuid,nodev,noexec,relatime,nsdelegate,memory_recursiveprot)
proc on /proc/sys type proc (ro,nosuid,nodev,noexec,relatime)
proc on /proc/sysrq-trigger type proc (ro,nosuid,nodev,noexec,relatime)
tmpfs on /sys/firmware type tmpfs (ro,relatime)


In [3]:
#attempt to remount with write permissions
!mount -o remount,rw /sys
!mount -o remount,rw /sys/fs/cgroup

In [4]:
#check cgroup again for write privelages "rw"
!mount | grep /sys

sysfs on /sys type sysfs (rw,nosuid,nodev,noexec,relatime)
cgroup on /sys/fs/cgroup type cgroup2 (rw,nosuid,nodev,noexec,relatime,nsdelegate,memory_recursiveprot)
proc on /proc/sys type proc (ro,nosuid,nodev,noexec,relatime)
proc on /proc/sysrq-trigger type proc (ro,nosuid,nodev,noexec,relatime)
tmpfs on /sys/firmware type tmpfs (ro,relatime)


In [5]:
#create a new cgroup
!mkdir /sys/fs/cgroup/my_cgroup
!ls /sys/fs/cgroup

cgroup.controllers	cpuset.cpus.effective	  hugetlb.2MB.max	    memory.pressure
cgroup.events		cpuset.cpus.partition	  hugetlb.2MB.numa_stat     memory.reclaim
cgroup.freeze		cpuset.mems		  hugetlb.2MB.rsvd.current  memory.stat
cgroup.kill		cpuset.mems.effective	  hugetlb.2MB.rsvd.max	    memory.swap.current
cgroup.max.depth	cpu.stat		  io.max		    memory.swap.events
cgroup.max.descendants	cpu.weight		  io.pressure		    memory.swap.high
cgroup.pressure		cpu.weight.nice		  io.stat		    memory.swap.max
cgroup.procs		hugetlb.1GB.current	  memory.current	    my_cgroup
cgroup.stat		hugetlb.1GB.events	  memory.events		    pids.current
cgroup.subtree_control	hugetlb.1GB.events.local  memory.events.local	    pids.events
cgroup.threads		hugetlb.1GB.max		  memory.high		    pids.max
cgroup.type		hugetlb.1GB.numa_stat	  memory.low		    pids.peak
cpu.idle		hugetlb.1GB.rsvd.current  memory.max		    rdma.current
cpu.max			hugetlb.1GB.rsvd.max	  memory.min		    rdma.max
cpu.max.burst		hugetlb.2MB.curre

In [6]:
#attempt to modify the cgroup
!echo 512 > /sys/fs/cgroup/my_cgroup/cpu.shares

/bin/bash: line 1: /sys/fs/cgroup/my_cgroup/cpu.shares: Permission denied


## **Namespace limitations**

**Similar to cgroups, within a Google Colab session, namespaces can be created, but there is a lack of privileges to modify and manage those namespaces fully. For example, a network namespace can be created using the `unshare` command. However, when attempting to bring the network interface up (such as the loopback interface) using `ip link set lo up`, we receive an error stating that the operation is not permitted. This limitation arises because the necessary administrative capabilities are missing. Docker requires the ability to manage and control network interfaces within network namespaces to allow containers to communicate with the host and other containers. Without the ability to perform these operations, Docker cannot function properly in this environment.**

In [7]:
#create a new network namespace
!unshare --net bash -c "ip link"

1: lo: <LOOPBACK> mtu 65536 qdisc noop state DOWN mode DEFAULT group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00


In [8]:
#attempt to bring the newly created network namespace up
!ip link set lo up

RTNETLINK answers: Operation not permitted


## **Attempting to Install Docker**

In [9]:
!sudo apt update
!sudo apt install docker.io

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,068 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
G

In [10]:
#attempt to start Docker
!service docker start

docker: unrecognized service


In [11]:
#attempt to pull/run the Docker hello-world image
!docker run hello-world

docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


# **udocker**

**An alternative to Docker is udocker. udocker is a user-space tool that enables running Docker containers in environments where users lack root privileges, such as Linux batch systems, HPC clusters and interactive systems. It allows non-privileged users to download, create, and execute containers without admin privileges. udocker acts as a wrapper around various technologies like PRoot, Fakechroot, runc, and Singularity to mimic basic Docker functionalities. Unlike Docker, which relies on kernel-level features such as namespaces and cgroups for isolation and resource management, udocker simulates root privileges using user namespaces and ptrace system calls via PRoot or Fakechroot. However, udocker does not offer the robust container isolation or system-level integration that Docker provides, making it suitable for environments where Docker cannot be used but with certain security and functionality limitations.**

<sup>Source: [udocker user manual Documentation](https://indigo-dc.github.io/udocker/user_manual.html)</sup>

## **Installing udocker**

**One way to install udocker is with the `pip` package manager for Python**

<sup>Source: [udocker Installation Documentation](https://indigo-dc.github.io/udocker/installation_manual.html)</sup>

In [12]:
!pip install udocker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.6/119.6 kB 4.8 MB/s eta 0:00:00


## **Pulling an Image with udocker**

**By default udocker pulls images from Dockerhub. In this section we will pull the [Golang (Go)](https://hub.docker.com/_/golang) image from Dockerhub and run a simple Go script that will print "Hello World" using udocker.**

In [13]:
!whoami

root


In [14]:
#cannot run udocker as root
!udocker pull golang

Error: do not run as root !


In [15]:
#create non-root user to run udocker
!adduser --disabled-password --gecos '' newuser

Adding user `newuser' ...
Adding new group `newuser' (1000) ...
Adding new user `newuser' (1000) with group `newuser' ...
Creating home directory `/home/newuser' ...
Copying files from `/etc/skel' ...


In [16]:
#pull a Docker image to work with
!sudo -u newuser udocker pull golang

Info: creating repo: /home/newuser/.udocker
Info: udocker command line interface 1.3.17
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools successful
Info: downloading layer sha256:17f4a873f3508262a7dfb2924949f7f9056ad3001bca21453453fdba5cdfe8af
Info: downloading layer sha256:4f4fb700ef54461cfa02571ae0db9a0dc1e0cdb5577484a6d75e68dc38e8acc1
Info: downloading layer sha256:f29d27dc8b2154be83283c4791da25b0264a9dc8484dfad644d065af56c19b72
Info: downloading layer sha256:2ac1f1163629431c9f488c4d6ff6afb5c73021839723b50bafe245663ad3d9df
Info: downloading layer sha256:b09bd0f8c72b1f7a4ca636b5aedb92a1eabce22f4608ea90f57daea57e43663a
Info: downloading layer sha256:7aadc5092c3b7a865666b14bef3d4d038282b19b124542f1a158c98ea8c1ed1b
Info: downloading layer sha256:da802df85c965baeca9d39869f9e2cbb3dc844d4627f413bfbb2f2c3d6055988
Info: downloading layer sha256:7d98d813d54f6207a57721008a4081378343ad8f1b2db66c121406019171805b


## **Running udocker**

In [17]:
#run command
!sudo -u newuser udocker run golang go version

 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING e111c88a-270b-3d1d-8b4f-f8a4a0c680b4                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: go
go version go1.23.2 linux/amd64


In [18]:
#create a simple Go script
%%bash
cat > hello.go
package main

import "fmt"

func main() {
    fmt.Println("Hello, World!")
}

In [19]:
#run the Go script with udocker
!sudo -u newuser udocker run --volume=$(pwd):/workspace -w=/workspace golang go run hello.go

 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING 2b2abd32-7fd8-304d-94df-79480755e1ae                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: go
Hello, World!


## **Issues with udocker in Google Colab**

**One of the issues with running udocker in Google Colab is that it can be slow. For example, printing "Hello World" with udocker and Go takes 40 seconds on average, while printing "Hello World" with the Go binary in Google Colab takes 250 milliseconds on average. This makes Go installed directly more than 1,000 times faster than when run through udocker. The reason for this slowdown is the emulation layer used by udocker, which is inherently slower than if udocker had direct system access.**

**To work with udocker, a non-root user needs to be created. The commands to run a udocker container can be relatively verbose compared to Docker commands run in a dedicated machine.**

**Because networking within Google Colab is restricted, there are images that cannot be utilized within Google Colab. For example the image for [Jupyter Notebook](https://hub.docker.com/r/jupyter/datascience-notebook) cannot be run because it runs a web server and requires opening ports for the notebook, which is not allowed in a Google Colab session.**

# **References and Additional Learning**

## **Docker Image**

- **[Golang Image](https://hub.docker.com/_/golang) from Dockerhub**

## **Documentation**

- **[CGROUPS](https://www.kernel.org/doc/Documentation/cgroup-v1/cgroups.txt) from kernel.org**

- **[Docker Engine overview](https://docs.docker.com/engine/) from docs.docker.com**

- **[dockerd](https://docs.docker.com/reference/cli/dockerd/) from docs.docker.com**

- **[Runtime metrics](https://docs.docker.com/engine/containers/runmetrics/) from docs.docker.com**

- **[udocker Installation Documentation](https://indigo-dc.github.io/udocker/installation_manual.html)**

- **[udocker user manual Documentation](https://indigo-dc.github.io/udocker/user_manual.html)**

- **[What is Docker?](https://docs.docker.com/get-started/docker-overview/) from docs.docker.com**

# **Connect**
- **Feel free to connect with Adrian on [YouTube](https://www.youtube.com/channel/UCPuDxI3xb_ryUUMfkm0jsRA), [LinkedIn](https://www.linkedin.com/in/adrian-dolinay-frm-96a289106/), [X](https://twitter.com/DolinayG), [GitHub](https://github.com/ad17171717), [Medium](https://adriandolinay.medium.com/) and [Odysee](https://odysee.com/@adriandolinay:0). Happy coding!**

# **Podcast**

- **Check out Adrian's Podcast, The Aspiring STEM Geek on [YouTube](https://www.youtube.com/@AdrianDolinay/podcasts), [Spotify](https://open.spotify.com/show/60dPNJbDPaPw7ru8g5btxV), [Apple Podcasts](https://podcasts.apple.com/us/podcast/the-aspiring-stem-geek/id1765996824), [Audible](https://www.audible.com/podcast/The-Aspiring-STEM-Geek/B0DC73S9SN?eac_link=MCFKvkxuqKYU&ref=web_search_eac_asin_1&eac_selected_type=asin&eac_selected=B0DC73S9SN&qid=IrZ84nGqvz&eac_id=141-8769271-5781515_IrZ84nGqvz&sr=1-1) and [iHeart Radio](https://www.iheart.com/podcast/269-the-aspiring-stem-geek-202676097/)!**